# Eat Safe, Love

## Notebook Set Up

In [12]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint
import pandas as pd

In [2]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [3]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [4]:
# review the collections in our database
db.list_collection_names()

['establishments']

In [5]:
# assign the collection to a variable
establishments = db['establishments']

## Part 3: Exploratory Analysis
Unless otherwise stated, for each question: 
* Use `count_documents` to display the number of documents contained in the result.
* Display the first document in the results using `pprint`.
* Convert the result to a Pandas DataFrame, print the number of rows in the DataFrame, and display the first 10 rows.

### 1. Which establishments have a hygiene score equal to 20?

In [10]:
# Find the establishments with a hygiene score of 20
query = {'scores.Hygiene': 20}
fields= {'BusinessName':1, 'scores.Hygiene':1}

results=establishments.find(query, fields)

# Use count_documents to display the number of documents in the result

count= establishments.count_documents(query)

print("Number of documents in result:", count)
print()

# Display the first document in the results using pprint


pprint(results[0])

Number of documents in result: 41

{'BusinessName': 'The Chase Rest Home',
 '_id': ObjectId('645fea83d561b5c3d002622a'),
 'scores': {'Hygiene': 20}}


In [13]:
# Convert the result to a Pandas DataFrame
results_df= pd.DataFrame(list(results))

# Display the number of rows in the DataFrame

print("The number of rows are:", len(results_df))

# Display the first 10 rows of the DataFrame
results_df.head(10)


The number of rows are: 41


,_id,BusinessName,scores
0,645fea83d561b5c3d002622a,The Chase Rest Home,{'Hygiene': 20}
1,645fea83d561b5c3d00265b3,Brenalwood,{'Hygiene': 20}
2,645fea83d561b5c3d00268c3,Melrose Hotel,{'Hygiene': 20}
3,645fea83d561b5c3d0026aae,Seaford Pizza,{'Hygiene': 20}
4,645fea83d561b5c3d0026abf,Golden Palace,{'Hygiene': 20}
5,645fea83d561b5c3d002745e,Ashby's Butchers,{'Hygiene': 20}
6,645fea83d561b5c3d002767e,South Sea Express Cuisine,{'Hygiene': 20}
7,645fea83d561b5c3d0028ba9,Golden Palace,{'Hygiene': 20}
8,645fea83d561b5c3d0029001,The Tulip Tree,{'Hygiene': 20}
9,645fea84d561b5c3d00297fb,F & S,{'Hygiene': 20}


### 2. Which establishments in London have a `RatingValue` greater than or equal to 4?

In [17]:
# Find the establishments with London as the Local Authority and has a RatingValue greater than or equal to 4.
query = {'LocalAuthorityName': 
                                {'$regex':'London'},
                                                        'RatingValue': {'$gte': 4}}

field= {'BusinessName': 1, 'LocalAuthorityName':1, 'RatingValue':1}

results= establishments.find(query, field)

# Use count_documents to display the number of documents in the result

count = establishments.count_documents(query)
print("Number of documents in result:", count)
print()

# Display the first document in the results using pprint
pprint(results[0])

Number of documents in result: 33

{'BusinessName': "Charlie's",
 'LocalAuthorityName': 'City of London Corporation',
 'RatingValue': 4,
 '_id': ObjectId('645fea83d561b5c3d0027c51')}


In [18]:
# Convert the result to a Pandas DataFrame

london_df= pd.DataFrame(list(results))

# Display the number of rows in the DataFrame
print("The number of rows are:", len(london_df))

# Display the first 10 rows of the DataFrame
london_df.head(10)

The number of rows are: 33


,_id,BusinessName,RatingValue,LocalAuthorityName
0,645fea83d561b5c3d0027c51,Charlie's,4,City of London Corporation
1,645fea83d561b5c3d0027f72,Mv City Cruises Erasmus,5,City of London Corporation
2,645fea83d561b5c3d0028ac4,Benfleet Motor Yacht Club,4,City of London Corporation
3,645fea84d561b5c3d00298c1,Coombs Catering t/a The Lock and Key,5,City of London Corporation
4,645fea84d561b5c3d00298ce,Tilbury Seafarers Centre,5,City of London Corporation
5,645fea84d561b5c3d0029fd3,Mv Valulla,5,City of London Corporation
6,645fea84d561b5c3d002c0d7,Tereza Joanne,5,City of London Corporation
7,645fea84d561b5c3d002c4a2,Brick Lane Brews,4,City of London Corporation
8,645fea84d561b5c3d002c4ba,The Nuance Group (UK) Limited,5,City of London Corporation
9,645fea84d561b5c3d002c4bb,WH Smith,5,City of London Corporation


### 3. What are the top 5 establishments with a `RatingValue` rating value of 5, sorted by lowest hygiene score, nearest to the new restaurant added, "Penang Flavours"?

In [30]:
# Search within 0.01 degree on either side of the latitude and longitude.
# Rating value must equal 5
# Sort by hygiene score

query_one = {'BusinessName': 'Penang Flavours'}

fields = {'BusinessName': 1,'geocode.latitude': 1, 'geocode.longitude': 1}
result_one = establishments.find(query_one, fields)

degree_search = 0.01
latitude = result_one[0]['geocode']['latitude']
longitude = result_one[0]['geocode']['longitude']


# Find Top 5 
query = {'RatingValue': 5,
         'geocode.latitude': {'$gte': (latitude - degree_search)},
         'geocode.latitude': {'$lte': (latitude + degree_search)},
         'geocode.longitude': {'$gte': (longitude - degree_search)},
         'geocode.longitude': {'$lte': (longitude + degree_search)},
         'scores.Hygiene': {'$ne': None}
         }



show = {'BusinessName': 1,'RatingValue': 1, 'scores.Hygiene':1, 'geocode.latitude': 1, 'geocode.longitude': 1}

sort=[('scores.Hygiene', 1)]

limit=5

result = establishments.find(query, show).sort(sort).limit(limit)


# Print the results
# for i in range (5):
#     pprint(result[i])

BusinessName_list=[]
RatingValue_list=[]
id_list=[]
latitude_list=[]
longitude_list=[]
hygiene_list=[]

for row in result:
    pprint(row)
    BusinessName= row['BusinessName']
    RatingValue= row['RatingValue']
    id= row["_id"]
    latitude= row['geocode']['latitude']
    longitude= row['geocode']['longitude']
    hygiene = row ['scores']['Hygiene']
    
    BusinessName_list.append(BusinessName)
    RatingValue_list.append(RatingValue)
    id_list.append(id)
    latitude_list.append(latitude)
    longitude_list.append(longitude)
    hygiene_list.append(hygiene)



{'BusinessName': 'Angies Newsagents',
 'RatingValue': 5,
 '_id': ObjectId('645fea83d561b5c3d0026c14'),
 'geocode': {'latitude': 50.77601, 'longitude': 0.093059},
 'scores': {'Hygiene': 0}}
{'BusinessName': 'Ab Fab Rooms',
 'RatingValue': 5,
 '_id': ObjectId('645fea83d561b5c3d0026ced'),
 'geocode': {'latitude': 50.780751, 'longitude': 0.083468},
 'scores': {'Hygiene': 0}}
{'BusinessName': 'Beachlands Care Home',
 'RatingValue': 5,
 '_id': ObjectId('645fea83d561b5c3d0026c69'),
 'geocode': {'latitude': 50.775392, 'longitude': 0.088903},
 'scores': {'Hygiene': 0}}
{'BusinessName': 'Costcutter Claremont',
 'RatingValue': 5,
 '_id': ObjectId('645fea83d561b5c3d0026c09'),
 'geocode': {'latitude': 50.77601, 'longitude': 0.093059},
 'scores': {'Hygiene': 0}}
{'BusinessName': 'Clifden House Dementia Care Centre',
 'RatingValue': 5,
 '_id': ObjectId('645fea83d561b5c3d0026c65'),
 'geocode': {'latitude': 50.776112, 'longitude': 0.089886},
 'scores': {'Hygiene': 0}}


In [38]:
# Convert result to Pandas DataFrame

RatingValue_df = pd.DataFrame({
    
 'Business Name': BusinessName_list,
 "Rating Value" : RatingValue_list,
    "id": id_list,
    "latitude" : latitude_list,
    "longitude" : longitude_list,
    "Hygiene" : hygiene_list

})

RatingValue_df.head()

,Business Name,Rating Value,id,latitude,longitude,Hygiene
0,Angies Newsagents,5,645fea83d561b5c3d0026c14,50.776010,0.093059,0
1,Ab Fab Rooms,5,645fea83d561b5c3d0026ced,50.780751,0.083468,0
2,Beachlands Care Home,5,645fea83d561b5c3d0026c69,50.775392,0.088903,0
3,Costcutter Claremont,5,645fea83d561b5c3d0026c09,50.776010,0.093059,0
4,Clifden House Dementia Care Centre,5,645fea83d561b5c3d0026c65,50.776112,0.089886,0


### 4. How many establishments in each Local Authority area have a hygiene score of 0?

In [39]:
establishments.find_one()

{'_id': ObjectId('645fea82d561b5c3d00246f9'),
 'FHRSID': 647177,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'PostCode': 'CT19 6PY',
 'Phone': '',
 'RatingValue': 4,
 'RatingKey': 'fhrs_4_en-gb',
 'RatingDate': '2014-03-31T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 10},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': 1.196408, 'latitude': 51.086058},
 'RightToReply': '',
 'Distance': 4591.821311183521,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  'itemCo

In [41]:
# Create a pipeline that: 
# 1. Matches establishments with a hygiene score of 0
# 2. Groups the matches by Local Authority
# 3. Sorts the matches from highest to lowest

pipeline = [
        {"$match": {"scores.Hygiene": 0}},
        {"$group": {"_id": "$LocalAuthorityName", "count": {"$sum": 1}}},
         {"$sort": {"count": -1}}
         
         ]



# Print the number of documents in the result

results = list(establishments.aggregate(pipeline))

# Print the first 10 results

pprint(results[0:10])


[{'_id': 'Thanet', 'count': 1130},
 {'_id': 'Greenwich', 'count': 882},
 {'_id': 'Maidstone', 'count': 713},
 {'_id': 'Newham', 'count': 711},
 {'_id': 'Swale', 'count': 686},
 {'_id': 'Chelmsford', 'count': 680},
 {'_id': 'Medway', 'count': 672},
 {'_id': 'Bexley', 'count': 607},
 {'_id': 'Southend-On-Sea', 'count': 586},
 {'_id': 'Tendring', 'count': 542}]


In [43]:
# Convert the result to a Pandas DataFrame
result_df= pd.DataFrame(results)
# Display the number of rows in the DataFrame

print("Number of rows in a DataFrame: ", len(result_df))

# Display the first 10 rows of the DataFrame

result_df.head(10)

Number of rows in a DataFrame:  55


,_id,count
0,Thanet,1130
1,Greenwich,882
2,Maidstone,713
3,Newham,711
4,Swale,686
5,Chelmsford,680
6,Medway,672
7,Bexley,607
8,Southend-On-Sea,586
9,Tendring,542
